# Setup

In [ ]:
import numpy as np

import seaborn as sns
import pandas as pd
import os
import pprint
import ast
import re
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import json

pp = pprint.PrettyPrinter(depth=6)

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

ALLOWED_METAL_NAMES =   {'Iron': 'FE+2', 'Cobalt': 'CO+2', 'Copper': 'CU+2', 'Manganese': 'MN+2', 'Molybdenum': 'CPD-8123', 'Nickel': 'NI+2', 'Zinc': 'ZN+2',
                        'Calcium': 'CA+2', 'Magnesium': 'MG+2', 'Sodium': 'NA+', 'Potassium': 'K+',
                        'Iron-sulfur \(4Fe-4S\)': 'CPD-7', 'Iron-sulfur \(2Fe-2S\)': 'CPD-6',
                         'Iron-sulfur \(4Fe-4S-S-AdoMet\)': 'CPD-7', 'Iron-sulfur \(3Fe-4S\)': '3FE-4S', 'Iron-oxo-sulfur \(4Fe-2O-2S\)': 'CPD-7',
                         'Iron-sulfur': 'CPD-7', # has to be after others since it is a substring of others
                        'heme': 'Heme-b', 'Molybdate': 'CPD-3', 'heme B': 'Heme-b', 'Cobalamin': 'COB-I-ALAMIN',
                         'Selenocysteine': 'L-SELENOCYSTEINE',
                        'Divalent metal cation': 'Any+2'}


ACCEPTED_OTHER_FEATURES = {'PYRIDOXAL_PHOSPHATE', 'THIAMINE-PYROPHOSPHATE', 'FMN', 'FAD', 'LIPOIC-ACID', 'BIOTIN'}

AMINO_ACID_MAP = {'A': 'ALA', 'C': 'CYS', 'D': 'ASP', 'E': 'GLU', 'F': 'PHE', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
                  'K': 'LYS', 'L': 'LEU', 'M': 'MET', 'N': 'ASN', 'P': 'PRO', 'Q': 'GLN', 'R': 'ARG', 'S': 'SER',
                  'T': 'THR', 'V': 'VAL', 'W': 'TRP', 'Y': 'TYR', 'U': 'SEL', '*': 'TER'}

# Create a list of temporarily allowed Gene Ontology terms to fix gaps in pathway annotations. Usually non-metabolic
# Currently: Tx Reg, Transcription, translation, DNA replication, Cell division, iron-sulfur cluster assembly, proteolysis, dna repair, copper response
TEMP_GO_TERMS = {'GO:0006355': 'Regulation of transcription', 'GO:0010468': 'Regulation of transcription',
                  'GO:0006351': 'Transcription', 'GO:0006350': 'Transcription',
                  'GO:0006412': 'Translation', 'GO:0006260': 'DNA replication', 'GO:0006457': 'Protein folding', 'GO:0045454': 'Redox homeostasis',
                  'GO:0051301': 'Cell division', 'GO:0015288': 'Porin',
                  'GO:0016226': 'Iron-sulfur cluster assembly', 'GO:0006508': 'Proteolysis', 'GO:0006281': 'DNA repair',
                  'GO:0046688': 'Response to copper ion'}


# as residues
AMINO_ACID_RESIDUE_MASSES = {
    'ALA': 71.03711, 'ARG': 156.10111, 'ASN': 114.04293, 'ASP': 115.02694,
    'CYS': 103.00919, 'GLU': 129.04259, 'GLN': 128.05858, 'GLY': 57.02146,
    'HIS': 137.05891, 'ILE': 113.08406, 'LEU': 113.08406, 'LYS': 128.09496,
    'MET': 131.04049, 'PHE': 147.06841, 'PRO': 97.05276, 'SER': 87.03203,
    'THR': 101.04768, 'TRP': 186.07931, 'TYR': 163.06333, 'VAL': 99.06841,
    'SEL': 150.0379, 'TER': 0.0
}


erroneous_monomer_metal_interactions = ["EG11415-MONOMER", "EG12132-MONOMER",
    "EG12332-MONOMER", "EG11378-MONOMER", "G7748-MONOMER", "CRR-MONOMER", "EG11663-MONOMER", "GLYOXI-MONOMER"]

def get_pathway_ith_level_parents(cur_pathway_idx, pathway_matrix, name_list, level_vector, level=2, parent_dict=None):

    if parent_dict is None:
        parent_dict = {}

    cur_pathway_level = level_vector[cur_pathway_idx]

    if cur_pathway_level == level:
        parent_dict[name_list[cur_pathway_idx]] = cur_pathway_level

    parent_slice = pathway_matrix[:, cur_pathway_idx]
    parent_idxs = np.where(parent_slice != 0)[0]


    for idx in parent_idxs:

        _ = get_pathway_ith_level_parents(idx, pathway_matrix, name_list, level_vector, level, parent_dict)

    return parent_dict

## Reload data

In [ ]:
parsed_complex_df = pd.read_csv('notebooks/cofactors/data/raw_complexes.csv', index_col=False)

# read stoichiometry, cofactors and enzyme_reaction as literal sets
for column in ['stoichiometry', 'cofactors', 'enzyme_reaction']:
    parsed_complex_df[column] = parsed_complex_df[column].apply(ast.literal_eval)

parsed_protein_df = pd.read_csv('notebooks/cofactors/data/raw_proteins.csv', index_col=False)

for column in ['cofactors', 'enzyme_reaction', 'metal_features', 'other_features', 'direct_annotations', 'go_annotations']:
    parsed_protein_df[column] = parsed_protein_df[column].apply(ast.literal_eval)


parsed_cofactor_df = pd.read_csv('notebooks/cofactors/data/raw_cofactors.csv', index_col=False)

for column in ['elemental_composition']:
    parsed_cofactor_df[column] = parsed_cofactor_df[column].apply(ast.literal_eval)

parsed_pathway_df = pd.read_csv('notebooks/cofactors/data/raw_pathways.csv', index_col=False)

for column in ['parents', 'children']:
    parsed_pathway_df[column] = parsed_pathway_df[column].apply(ast.literal_eval)

# add GO "pathways" to pathway df
go_collection = []
go_collection.append({'id': 'Other functions', 'common_name': 'Other functions', 'level': 1, 'parents': [], 'children': []})

for go_term, go_name in TEMP_GO_TERMS.items():
    go_collection[0]['children'].append(go_name)
    go_collection.append({'id': go_name, 'common_name': go_name, 'level': 2, 'parents': ['Other functions'], 'children': []})

go_df = pd.DataFrame(go_collection)

parsed_pathway_df = pd.concat([parsed_pathway_df, go_df], ignore_index=True)
parsed_pathway_df

# Data processing into final tables
## Specific adjustments

In [ ]:
# removing specific troublesome interactions that have been discovered in the data
# should not be in model


# remove metal features from EG11415-MONOMER in parsed_protein_df\
for protein in erroneous_monomer_metal_interactions:
    prot_idx = np.where(parsed_protein_df['id'] == protein)[0][0]
    parsed_protein_df.at[prot_idx, 'metal_features'] = []

# classify folE gene use as cofactor production (THF)
pathway_idx = parsed_pathway_df[parsed_pathway_df['id'] == '6-HM-Dihydropterin-PP-Biosynthesis'].index[0]
parsed_pathway_df.at[pathway_idx, 'parents'] = ['Cofactor-Biosynthesis']

protein_idx = parsed_protein_df[parsed_protein_df['id'] == 'GTP-CYCLOHYDRO-I-MONOMER'].index[0]
parsed_protein_df.at[protein_idx, 'direct_annotations'] = set(['PWY-6147'])

In [ ]:
set(['PWY-6147'])

## Process raw EcoCyc annotations into standard EcoCyc names

In [ ]:

# remove all \ characters from keys in ALLOWED_METAL_NAMES
NON_REGEX_METAL = {key.replace('\\', ''): value for key, value in ALLOWED_METAL_NAMES.items()}

parsed_protein_df['metal_features_processed'] = 0
parsed_protein_df['metal_features_processed'] = parsed_protein_df['metal_features_processed'].astype(object)

metal_pattern = '|'.join(ALLOWED_METAL_NAMES.keys())
metal_regex = re.compile(f'(({metal_pattern})(\s\d[\.,;]|[\.,;]|\s\())')


for i in range(len(parsed_protein_df.index)):

    metal_binding = parsed_protein_df.loc[i, 'metal_features']

    metal_count_dict = {}
    existing_matches = set()

    for feature in metal_binding:
        matches = metal_regex.search(feature)
        if matches:
            metal = matches.group(0)[:-1]

            # eliminate duplicates
            if metal not in existing_matches:

                existing_matches.add(metal)

                if 'heme' in feature:
                    metal = metal.replace('Iron', 'heme')
                if 'alamin' in feature:
                    metal = metal.replace('Cobalt', 'Cobalamin')

                # check if last char of metal is a number, then crop
                if metal[-1].isdigit():
                    metal = metal[:-2]

                metal = metal.strip()

                # replace metal name with allowed metal name
                metal = NON_REGEX_METAL[metal]

                if metal in metal_count_dict:
                    metal_count_dict[metal] += 1
                else:
                    metal_count_dict[metal] = 1

        else:
            print(f'No match for {feature} in {parsed_protein_df.loc[i, "id"]}')



    # EXCEPTIONS
    # if both magnesium and manganese are present, replace with magnesium
    # TODO remove when using UniProt data. Ecocyc data is not as reliable
    if 'MG+2' in metal_count_dict and 'MN+2' in metal_count_dict and metal_count_dict['MG+2'] == metal_count_dict['MN+2']:
        del metal_count_dict['MN+2']
    # same with cobalt
    if 'CO+2' in metal_count_dict and 'MG+2' in metal_count_dict and metal_count_dict['CO+2'] == metal_count_dict['MG+2']:
        del metal_count_dict['CO+2']
    elif 'CO+2' in metal_count_dict:
        metal_count_dict['MG+2'] = metal_count_dict['CO+2']
        del metal_count_dict['CO+2']

    parsed_protein_df.at[i, 'metal_features_processed'] = metal_count_dict

In [ ]:
# manual corrections in data
correction_idx = parsed_protein_df.index[parsed_protein_df['id'] == '3-OXOACYL-ACP-REDUCT-MONOMER'][0]

# remove cofactors
parsed_protein_df.at[correction_idx, 'metal_features_processed'] = {}

correction_idx = parsed_protein_df.index[parsed_protein_df['id'] == 'CARBPSYN-LARGE'][0]

# remove cofactors
parsed_protein_df.at[correction_idx, 'metal_features_processed']
mn_cofactor_count = parsed_protein_df.at[correction_idx, 'metal_features_processed']['MN+2']
parsed_protein_df.at[correction_idx, 'metal_features_processed'] = {'MG+2': mn_cofactor_count}

# dps
correction_idx = parsed_protein_df.index[parsed_protein_df['id'] == 'EG11415-MONOMER'][0]
parsed_protein_df.at[correction_idx, 'metal_features_processed'] = {}

In [ ]:
parsed_protein_df = parsed_protein_df.drop(columns=['metal_features'])
parsed_protein_df

In [ ]:
parsed_protein_df['other_features_processed'] = 0
parsed_protein_df['other_features_processed'] = parsed_protein_df['other_features_processed'].astype(object)

for i in range(len(parsed_protein_df.index)):

    other_features = parsed_protein_df.loc[i, 'other_features']

    other_feature_count_dict = {}
    existing_matches = set()

    for feature in other_features:

        # eliminate duplicates
        if feature not in existing_matches:

            existing_matches.add(feature)

            if feature in ACCEPTED_OTHER_FEATURES:
                if feature in other_feature_count_dict:
                    other_feature_count_dict[feature] += 1
                else:
                    other_feature_count_dict[feature] = 1

    parsed_protein_df.at[i, 'other_features_processed'] = other_feature_count_dict

In [ ]:
# processed go_annotations. if no pathways exist for monomer, use go annotations
for i in range(len(parsed_protein_df.index)):

    cur_go_annotations = parsed_protein_df.loc[i, 'go_annotations']
    cur_pathways = parsed_protein_df.loc[i, 'direct_annotations']

    replacement_pathways = list()

    if len(cur_pathways) == 0:

        for go_term in TEMP_GO_TERMS:
            for go_annotation in cur_go_annotations:
                if go_term == go_annotation:
                    replacement_pathways.append(TEMP_GO_TERMS[go_term])

        if len(replacement_pathways) > 0:
            print(parsed_protein_df.at[i, "id"], set([replacement_pathways[0]]))
            parsed_protein_df.at[i, 'direct_annotations'] = set([replacement_pathways[0]])


# parsed_protein_df[parsed_protein_df['id'] == 'PD00197']

In [ ]:
# decompose sequence into dict of amino acid counts
parsed_protein_df['sequence_processed'] = 0
parsed_protein_df['sequence_processed'] = parsed_protein_df['sequence_processed'].astype(object)

parsed_protein_df['sequence_mass'] = 0

unique_aa = set()

for i in range(len(parsed_protein_df.index)):

    sequence = parsed_protein_df.loc[i, 'seq']

    if type(sequence) != str:
        print(f'No sequence for {parsed_protein_df.loc[i, "id"]}')
        continue

    aa_count_dict = {}

    mass = 0

    for aa in sequence:
        if aa in aa_count_dict:
            aa_count_dict[aa] += 1
        else:
            aa_count_dict[aa] = 1

        if aa not in unique_aa:
            unique_aa.add(aa)

        mass += AMINO_ACID_RESIDUE_MASSES[AMINO_ACID_MAP[aa]]

    parsed_protein_df.at[i, 'sequence_processed'] = aa_count_dict
    parsed_protein_df.at[i, 'sequence_mass'] = mass

In [ ]:
parsed_protein_df = parsed_protein_df.drop(columns=['other_features','seq'])

parsed_protein_df

## Create new column for monomer component stoichiometry

In [ ]:
complex_ids = parsed_complex_df['id'].tolist()
monomer_names = parsed_protein_df['id'].tolist()

In [ ]:
def recursive_component_tree(current_component_name, complex_table, protein_table,
                             current_multiplier=1, component_list=None, parent=None, return_cofactors=False):
    """
    Recursively find all downstream components of a given complex.
    """

    complex_names = complex_table['id'].tolist()
    monomer_names = protein_table['id'].tolist()


    my_children = {}

    if component_list is None:
        component_list = []


    if current_component_name in complex_names:


        cplx_idx = complex_table.index[complex_table['id'] == current_component_name][0]
        stoichiometry = complex_table.at[cplx_idx, 'stoichiometry']

        direct_children = {k: abs(v) for k, v in stoichiometry.items() if v < 0}

        for component_name, coefficient in stoichiometry.items():

            if coefficient < 0 and component_name != current_component_name:

                child_multiplier = abs(coefficient * current_multiplier)

                new_child = recursive_component_tree(component_name, complex_table, protein_table,
                                                     child_multiplier, component_list, current_component_name, return_cofactors)

                my_children = my_children | new_child


            elif coefficient > 0 and component_name == current_component_name:
                continue

            else:
                raise ValueError(f"key {component_name} and value {coefficient} for complex {component_name} not processed properly.")

        component_list.append({'name': current_component_name,'parent': parent, 'children': direct_children,
                               'multiplier': int(current_multiplier), })


    elif current_component_name in monomer_names:

        # TODO check if enzrxn
        if return_cofactors:
            protein_idx = protein_table.index[protein_table['id'] == current_component_name][0]

            protein_metals = protein_table.at[protein_idx, 'metal_features_processed']
            protein_other = protein_table.at[protein_idx, 'other_features_processed']

            table_cofactors = protein_metals | protein_other

            if len(table_cofactors) > 0:
                # TODO Add apo protein to component list
                my_children = {}

                for cofactor, cofactor_coefficient in table_cofactors.items():
                    if table_cofactors[cofactor] !=  None:
                        my_children[cofactor] = cofactor_coefficient
                        component_list.append({'parent': current_component_name,
                                               'name': cofactor,
                                               'multiplier': abs(int(current_multiplier * cofactor_coefficient)),
                                               'children': None})

            component_list.append({'parent': parent, 'name': current_component_name, 'multiplier': current_multiplier, 'children': my_children})

        else:
            my_children = None
            component_list.append({'parent': parent, 'name': current_component_name, 'multiplier': current_multiplier, 'children': None})



    else:
        print(f"component {current_component_name} not found in complex or protein tables")

        return {}


    if parent is None:
        return {current_component_name: my_children}, component_list
    else:
        return {current_component_name: my_children}


In [ ]:
complex_tree_structure, nodes = recursive_component_tree('CPLX0-8167', parsed_complex_df, parsed_protein_df)
pp.pprint(nodes)

In [ ]:
parsed_complex_df['monomer_component_stoichiometry'] = 0
parsed_complex_df['monomer_component_stoichiometry'] = parsed_complex_df['monomer_component_stoichiometry'].astype(object)

for i in range(len(parsed_complex_df.index)):
    complex_name = parsed_complex_df.loc[i, 'id']
    complex_tree_structure, nodes = recursive_component_tree(complex_name, parsed_complex_df, parsed_protein_df)

    monomer_components = {node['name']: node['multiplier'] for node in nodes if node['children'] is None}

    parsed_complex_df.at[i, 'monomer_component_stoichiometry'] = monomer_components

In [ ]:
parsed_complex_df = parsed_complex_df.loc[:, ["id", "common_name", "stoichiometry", "monomer_component_stoichiometry", "cofactors"]]
parsed_complex_df

## Create tree matrix (also for Julia)

In [ ]:
# save names
complex_ids = list(parsed_complex_df['id'])
protein_ids = list(parsed_protein_df['id'])
cofactor_ids = list(parsed_cofactor_df['id'])
pathway_ids = list(parsed_pathway_df['id'])


name_idx = complex_ids + protein_ids + cofactor_ids
tree_matrix = np.zeros([len(complex_ids) + len(protein_ids) + len(cofactor_ids), len(complex_ids) + len(protein_ids) + len(cofactor_ids)], dtype=np.int64)

for i in range(len(parsed_complex_df)):
    name = parsed_complex_df.at[i, 'id']
    tree_structure, nodes = recursive_component_tree(name, parsed_complex_df, parsed_protein_df, return_cofactors=True)

    for node in nodes:
        node_name = node['name']
        node_children = node['children']

        if node_children != None:
            for child_name, child_coefficient in node_children.items():
                if child_name in name_idx:
                        tree_matrix[name_idx.index(node_name), name_idx.index(child_name)] = child_coefficient

# Create matrices to get cofactor counts

In [ ]:
parsed_pathway_df

In [ ]:
priority_list = ['Activation-Inactivation-Interconversion','Metabolic-Clusters', 'Macromolecule-Modification', 'Glycan-Pathways','Detoxification',  'Degradation']

# pathway matrix is necessary to traverse tree
pathway_matrix = np.zeros((len(pathway_ids), len(pathway_ids)), dtype=np.int64)
level_vector = np.zeros(len(pathway_ids), dtype=np.int64)

for i in range(len(parsed_pathway_df)):

    cur_pathway = parsed_pathway_df.at[i, 'id']
    level_vector[i] = parsed_pathway_df.at[i, 'level']

    pathway_parents = parsed_pathway_df.at[i, 'parents']
    pathway_children = parsed_pathway_df.at[i, 'children']

    for parent in pathway_parents:
        j = pathway_ids.index(parent)
        pathway_matrix[j, i] = 1

    for child in pathway_children:
        j = pathway_ids.index(child)
        pathway_matrix[i, j] = 1

original_pathway_matrix = pathway_matrix.copy()

# get superpathway indices
super_pathway_idx = pathway_ids.index('Super-Pathways')
super_pathway_children_idxs = np.where(pathway_matrix[super_pathway_idx, :] == 1)[0]

# zero out all superpathway children
pathway_matrix[:, super_pathway_children_idxs] = 0

# for columns (children) with multiple parents, if one parent leads to degradation or glycans, remove it.
for i in range(len(pathway_matrix[0, :])):
    cur_pathway = pathway_ids[i]

    if pathway_matrix[:, i].sum() > 1:

        nz_idxs = np.where(pathway_matrix[:, i] == 1)[0]
        top_level_classes = [list(get_pathway_ith_level_parents(j, original_pathway_matrix, pathway_ids, level_vector, level=1).keys())[0] for j in nz_idxs]
        # print(f"multiple parents {top_level_classes} for {cur_pathway}")


        # when there are multiple parents, remove them in the following order of priority:
        for priority in priority_list:
            while priority in top_level_classes and len(nz_idxs) > 1:
                priority_index = top_level_classes.index(priority)
                pathway_matrix[nz_idxs[priority_index], i] = 0
                nz_idxs = np.where(pathway_matrix[:, i] == 1)[0]
                top_level_classes[priority_index] = 'N/A'


        # then, if there are still multiple parents, remove all but the first one
        # TODO Change to parent with most frequently occuring 2nd parent.
        if len(nz_idxs) > 1:
            # top_two_level_classes = [list(get_pathway_ith_level_parents(j, original_pathway_matrix, pathway_name_list, level_vector, level=2).keys())[0] for j in nz_idxs]
            # print(f"multiple parents with 2nd level categories {top_two_level_classes} for {cur_pathway}")
            pathway_matrix[nz_idxs[1:], i] = 0

        nz_idxs = np.where(pathway_matrix[:, i] == 1)[0]
        top_level_classes = [list(get_pathway_ith_level_parents(j, original_pathway_matrix, pathway_ids, level_vector, level=1).keys()) for j in nz_idxs]

        # print(f"pruned to {top_level_classes}")

In [ ]:
# if choice can be made, don't pick these.
priority_list_second = ['SECONDARY-METABOLITE-BIOSYNTHESIS', 'Respiration']

# create protein name to pathway mapping
W = np.zeros((len(parsed_protein_df.index), len(parsed_pathway_df.index)))

for i in range(len(parsed_protein_df.index)):

    cur_pathways = parsed_protein_df.at[i, 'direct_annotations']

    for pathway in cur_pathways:
        pathway_idx = pathway_ids.index(pathway)
        W[i, pathway_idx] = 1


for i in range(len(parsed_protein_df.index)):

    cur_protein_pathways_idxs = np.where(W[i, :] == 1)[0]
    cur_protein = parsed_protein_df.at[i, 'id']

    if len(cur_protein_pathways_idxs) < 2:
        continue

    else:

        # for pathway_idx in cur_protein_pathways_idxs:

        # get top level class of every pathway, and remove all but the first one of each class.

        cur_protein_pathway_parents = list()
        cur_protein_pathway_two_parents = list()

        for pathway_idx in cur_protein_pathways_idxs:
            top_parents = get_pathway_ith_level_parents(pathway_idx, pathway_matrix, pathway_ids, level_vector, level=1)
            top_two_parents = get_pathway_ith_level_parents(pathway_idx, pathway_matrix, pathway_ids, level_vector, level=2)

            print(f"{cur_protein} has 2nd parents {top_two_parents} for {pathway_ids[pathway_idx]}")

            if len(top_parents) > 1:
                print(f"multiple parents {top_parents} for {pathway_ids[pathway_idx]} for {cur_protein}, should not happen.")

            if len(top_parents) == 1:
                cur_protein_pathway_parents.append(list(top_parents.keys())[0])
                cur_protein_pathway_two_parents.append(list(top_two_parents.keys())[0])
            else:
                # remove pathway with no parents
                W[i, pathway_idx] = 0
                cur_protein_pathway_parents.append('N/A')
                cur_protein_pathway_two_parents.append('N/A')


        # TODO - remove direct annotations with deprioritized parents
        for priority in priority_list:
            while priority in cur_protein_pathway_parents and len(np.where(W[i, :] == 1)[0]) > 1:
                priority_index = cur_protein_pathway_parents.index(priority)
                W[i, cur_protein_pathways_idxs[priority_index]] = 0
                cur_protein_pathway_parents[priority_index] = 'N/A'

        # same for 2nd level
        for priority in priority_list_second:
            while priority in cur_protein_pathway_two_parents and len(np.where(W[i, :] == 1)[0]) > 1:
                priority_index = cur_protein_pathway_two_parents.index(priority)
                W[i, cur_protein_pathways_idxs[priority_index]] = 0
                cur_protein_pathway_two_parents[priority_index] = 'N/A'

        new_protein_pathways_idxs = np.where(W[i, :] == 1)[0]

        # for pathway_idx in cur_protein_pathways_idxs:

        # get top level class of every pathway, and remove all but the first one of each class.

        new_protein_pathway_parents = list()
        new_protein_pathway_two_parents = list()

        for pathway_idx in new_protein_pathways_idxs:
            top_parents = get_pathway_ith_level_parents(pathway_idx, pathway_matrix, pathway_ids, level_vector, level=1)
            top_two_parents = get_pathway_ith_level_parents(pathway_idx, pathway_matrix, pathway_ids, level_vector, level=2)

            if len(top_parents) == 1:
                new_protein_pathway_parents.append(list(top_parents.keys())[0])
                new_protein_pathway_two_parents.append(list(top_two_parents.keys())[0])

        print(f"pruned to {cur_protein_pathway_two_parents}")


        # remove all N/A
        # cur_protein_pathways_idxs = cur_protein_pathways_idxs[cur_protein_pathway_parents != 'N/A']

        if len(np.unique(cur_protein_pathway_parents)) < 2:
            continue


In [ ]:
# create pathway to 2nd layer pathway mapping
W2 = np.zeros((len(parsed_pathway_df.index), len(parsed_pathway_df.index)))

for i in range(len(parsed_pathway_df.index)):

    cur_pathway = parsed_pathway_df.at[i, 'id']
    cur_pathway_idx = pathway_ids.index(cur_pathway)

    # get 2nd level parents
    cur_pathway_parents = get_pathway_ith_level_parents(cur_pathway_idx, pathway_matrix, pathway_ids, level_vector, level=2)

    # if len(cur_pathway_parents) > 1:
    #     print(f"cur pathway {cur_pathway} has parents {cur_pathway_parents}")

    for parent in cur_pathway_parents:
        parent_idx = pathway_ids.index(parent)
        W2[i, parent_idx] = 1

# zero diagonal (don't return self, since some pathways return themselves as level 2 parents)
np.fill_diagonal(W2[0:(W2.shape[0] - len(go_collection)), 0:(W2.shape[0] - len(go_collection))], 0)


W1 = np.zeros((len(parsed_pathway_df.index), len(parsed_pathway_df.index)))

for i in range(len(parsed_pathway_df.index)):

    if parsed_pathway_df.at[i, 'level'] <= 2:
        cur_pathway = parsed_pathway_df.at[i, 'id']
        cur_pathway_idx = pathway_ids.index(cur_pathway)

        # get 2nd level parents
        cur_pathway_parents = get_pathway_ith_level_parents(cur_pathway_idx, pathway_matrix, pathway_ids, level_vector, level=1)

        for parent in cur_pathway_parents:
            parent_idx = pathway_ids.index(parent)
            W1[i, parent_idx] = 1

np.fill_diagonal(W1, 0)



In [ ]:
# C matrix transforms complexes + monomers to just monomers.

# create protein name to index mapping
protein_name_to_index = {}
for i in range(len(parsed_protein_df.index)):
    protein_name_to_index[parsed_protein_df.at[i, 'id']] = i

# C matrix: complexes x proteins
C = np.zeros((len(parsed_complex_df.index), len(parsed_protein_df.index)))

for i in range(len(parsed_complex_df.index)):

    complex_components = parsed_complex_df.loc[i, 'monomer_component_stoichiometry']

    # TODO consider cofactors
    # complex_cofactors = filter_complex_df.loc[i, 'cofactors']


    for component_name, component_count in complex_components.items():
        if component_count is not None:             # side effect of parquet
            # get index of component in filter_protein_df
            component_index = protein_name_to_index[component_name]

            if parsed_complex_df.at[i, 'id'] == 'APORNAP-CPLX':
                print(f'component_name: {component_name}, component_count: {component_count}, component_index: {component_index}')

            C[i, component_index] = component_count

# append an identity matrix to C
C = np.concatenate((C, np.identity(len(parsed_protein_df.index))), axis=0)

C_names = list(parsed_complex_df['id']) + list(parsed_protein_df['id'])

In [ ]:
# P matrix transforms proteins to their respective cofactor counts.

# create cofactor name to index mapping
cofactor_name_to_index = {}
for i in range(len(parsed_cofactor_df.index)):
    cofactor_name_to_index[parsed_cofactor_df.at[i, 'id']] = i

cofactor_ids = list(parsed_cofactor_df['id'])

# P matrix: proteins x cofactors
P = np.zeros((len(parsed_protein_df.index), len(parsed_cofactor_df.index)))

for i in range(len(parsed_protein_df.index)):
    protein_metals = parsed_protein_df.loc[i, 'metal_features_processed']
    protein_other = parsed_protein_df.loc[i, 'other_features_processed']

    for metal, count in protein_metals.items():
        if count is not None:             # side effect of parquet
            cofactor_index = cofactor_name_to_index[metal]
            P[i, cofactor_index] = count

    for other, count in protein_other.items():
        if count is not None:             # side effect of parquet
            cofactor_index = cofactor_name_to_index[other]
            P[i, cofactor_index] = count


In [ ]:
# E matrix transforms cofactors to their respective elemental composition

# create list of unique elements
unique_elements = set()
for i in range(len(parsed_cofactor_df.index)):
    cofactor = parsed_cofactor_df.at[i, 'elemental_composition']
    unique_elements.update(cofactor.keys())

unique_elements = list(unique_elements)

# create E matrix: cofactors x elements
E = np.zeros((len(parsed_cofactor_df.index), len(unique_elements)))

for i in range(len(parsed_cofactor_df.index)):
    cofactor = parsed_cofactor_df.at[i, 'elemental_composition']

    for element, count in cofactor.items():
        if count is not None:             # side effect of parquet
            element_index = unique_elements.index(element)
            E[i, element_index] = count


element_ids = unique_elements

In [ ]:
# A matrix transforms proteins to their amino acid composition

A = np.zeros((len(parsed_protein_df.index), len(unique_aa)))

amino_acid_single_letter = list(unique_aa)
amino_acid_ids = [AMINO_ACID_MAP[aa] for aa in amino_acid_single_letter]

for i in range(len(parsed_protein_df.index)):
    protein = parsed_protein_df.at[i, 'sequence_processed']

    if type(protein) is not dict:
        continue

    for aa, count in protein.items():
        if count is not None:             # side effect of parquet
            aa_index = amino_acid_single_letter.index(aa)
            A[i, aa_index] = count

# Now ... add the counts >:o

In [ ]:
time = '3000'
date = '2023-10-31'
experiment = 'metabolism-redux-classic'
entry = f'{experiment}_{time}_{date}'
folder = f'out/cofactors/{entry}/'

In [ ]:
output_all = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output_all['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

fluxes = np.array(fba['estimated_fluxes'][1:])
exchanges = fba['estimated_exchange_dmdt']

# output['listeners']['unique_molecule_counts']['active_ribosome']

In [ ]:
output['listeners']['unique_molecule_counts']['active_RNAP']

In [ ]:
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry


In [ ]:
initial_state = json.load(open('data/wcecoli_t0.json'))

bulk_ids = [item[0] for item in initial_state['bulk']]

bulk.columns = bulk_ids

In [ ]:
# in the bulk dataframe, update RNAP and 50S, 30S rib counts from unique molecules since they are sequestered as unique when in use.
for unique_key, bulk_id in [('active_ribosome', 'CPLX0-3962'), ('active_ribosome', 'CPLX0-3953'), ('active_RNAP', 'APORNAP-CPLX')]:
    if unique_key in output['listeners']['unique_molecule_counts']:
        unique_count = output['listeners']['unique_molecule_counts'][unique_key]
        bulk.loc[:, bulk_id+'[c]'] += unique_count

In [ ]:
ecocyc_to_wcm_map = {}

# combined complex and protein names
complex_protein_names = list(parsed_protein_df['id']) + list(parsed_complex_df['id'])

for name in complex_protein_names:

    # find complex name in bulk_ids
    found = False

    try:
        idx = bulk_ids.index(name+'[c]')
        ecocyc_to_wcm_map[name] = name+'[c]'
        found = True
        # print(f'found {complex_name} at {idx}')

    except ValueError:
        # delete key
        found = False


    if found == False:

        for id in bulk_ids:
            if name+'[' in id and id.startswith(name) and bulk.loc[:, id].sum() > 0:
                #print(f'found {name} in {id} with nonzero count')
                ecocyc_to_wcm_map[name] = id
                found = True
                break           # ensures preferring nonzero counts

            elif name+'[' in id and id.startswith(name):
                # print(f'found {name} in {id} with zero count')
                ecocyc_to_wcm_map[name] = id
                found = True

    if found == False:
        ecocyc_to_wcm_map[name] = '--TRANS-ACENAPHTHENE-12-DIOL[c]' # should be none
        print(f'could not find {name}')


In [ ]:
complex_wcm_names = [ecocyc_to_wcm_map[name] for name in C_names]

counts = bulk.loc[:, complex_wcm_names]

In [ ]:
time = '1300'
date = '2023-12-14'
experiment = 'metabolism-redux-classic'
entry = f'{experiment}_{time}_{date}'
folder = f'out/cofactors/{entry}/'

In [ ]:
output_all = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output_all['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

fluxes = np.array(fba['estimated_fluxes'][1:])
exchanges = fba['estimated_exchange_dmdt']

bulk_ids = [item[0] for item in initial_state['bulk']]

bulk.columns = bulk_ids
# output['listeners']['unique_molecule_counts']['active_ribosome']

In [ ]:
# in the bulk dataframe, update RNAP and 50S, 30S rib counts from unique molecules since they are sequestered as unique when in use.
for unique_key, bulk_id in [('active_ribosome', 'CPLX0-3962'), ('active_ribosome', 'CPLX0-3953'), ('active_RNAP', 'APORNAP-CPLX')]:
    if unique_key in output['listeners']['unique_molecule_counts']:
        unique_count = output['listeners']['unique_molecule_counts'][unique_key]
        bulk.loc[:, bulk_id+'[c]'] += unique_count

In [ ]:
rich_counts = bulk.loc[:, complex_wcm_names]

# External data sets

# Get surface area occupancy estimates

In [ ]:
# roughly 90 by 35 rectangle. pretty big? Multiplicity 2.
# 0.009 x 0.0035 um^2 = 0.000032 um^2 * 3500 / 2 = 0.055 um^2
# in rich media
# 0.009 x 0.0035 um^2 = 0.000032 um^2 * 15000 / 2 = 0.24 um^2
counts.at[0, ecocyc_to_wcm_map['CPLX-157']]

In [ ]:
# 160 x 50 angstrom, = 0.016 x 0.005 um^2 = 0.00008 um^2 * 1000 = 0.08 um^2. Multiplicity 1
# in rich media
# 0.00008 um^2 * 3000 = 0.24 um^2, if 15000 would be like 20-30% of membrane space.
counts.at[0, ecocyc_to_wcm_map['NADH-DHI-CPLX']]

In [ ]:
# roughly 70 by 40 rectangle. pretty big?, 0.007 x 0.004 um^2 = 0.000028 um^2 * 3000 ~= 0.09 um^2
# grows to ~ 0.5 um^2 for 15000 counts in rich. Multiplicity 1. 8-15% of membrane space.
counts.at[0, ecocyc_to_wcm_map['CYT-O-UBIOX-CPLX']]

In [ ]:
# atpsyn, 25 rad circle, plus roughly 25 width accessory circling around. Multiplicity 1
# 0.005 um^2 * 3 = 0.000075 um^2 * 3500 = 0.26 um^2
# in rich media
# 0.005 um^2 * 3 = 0.000075 um^2 * 10000 = 0.75 um^2, 10-20% of membrane space.
counts.at[0, ecocyc_to_wcm_map['ATPSYN-CPLX']]

In [ ]:
# sdh, rough 37 angstrom radius, half filled circle. multiplicity 3
# 0.0037 um^2 * 3 / (2*3) = 0.000006845 um^2 * 3500 = 0.024 um^2
# in rich, about the same.

In [ ]:
# ftsh, roughly 100 angstrom radius, circle. Multiplicity 24. Small impact.
# 0.01 um^2 * 3 = 0.0003 um^2 * 1000/24 = 0.0125 um^2

In [ ]:
# secyeg, roughly 40x40 rectangle. Multiplicity 1.
# 0.004 um^2  = 0.000016 um^2 * 1700 = 0.0272 um^2
# in rich, increases sixfold to 0.16 um^2. Becomes around 2-5%. Not bad.

# add secdf


In [ ]:
# tcs, about 30-50 copies for 30 different systems. 8 angstrom radius for dimer. Multiplicity 2.
# 0.0008 um^2 * 3= 0.00000192 um^2 * 30 * 30 / 2  = 0.000900 um^2

In [ ]:
# mscS, 40 angstrom radius, circle. 3000 copies. Multiplicity 7.
# 0.004 um^2 * 3 = 0.000048 um^2 * 3000 / 7 = 0.0210 um^2
# 1-2% of membrane. Not bad.

# mscL, 20 angstrom radius, circle. 3000 copies. Multiplicity 5.
# 0.002 um^2 * 3 = 0.000012 um^2 * 3000 / 5 = 0.0072 um^2

In [ ]:
# pntAB, 60x20 rectangle. 3000-6000 copies, multiplicity 2.
# 0.006 * 0.002 = 0.000012 um^2 * 3000 / 2 = 0.018 um^2
# 1-2% of membrane. Not bad.

In [ ]:
# read in surface area occupancy estimates
area_df = pd.read_csv('notebooks/cofactors/external_data/membrane_areas.txt', skipinitialspace=True)
area_df

# strip whitespace from

In [ ]:
normal_surface_area = 4
rich_surface_area = 8

area_df['fraction_normal'] = (area_df['count_normal'] / area_df['multiplicity'] * area_df['estimated_area um2']) / normal_surface_area
area_df['fraction_rich'] = (area_df['count_rich'] / area_df['multiplicity'] * area_df['estimated_area um2']) / rich_surface_area
area_df

In [ ]:
area_df['fraction_normal'].sum()

In [ ]:
area_df['fraction_rich'].sum()

In [ ]:
area_df['fraction_rich']  / area_df['fraction_rich'].sum()

# Saving outputs to files compatible with Julia

In [ ]:
parsed_pathway_df

In [ ]:
complex_names = list(parsed_complex_df['common_name'])
protein_names = list(parsed_protein_df['common_name'])
cofactor_names = list(parsed_cofactor_df['common_name'])
pathway_names = list(parsed_pathway_df['common_name'])

# if cplx name is nan then use id
complex_names = [complex_names[i] if type(complex_names[i]) == str else parsed_complex_df['id'][i] for i in range(len(complex_names))]
protein_names = [protein_names[i] if type(protein_names[i]) == str else parsed_protein_df['id'][i] for i in range(len(protein_names))]
cofactor_names = [cofactor_names[i] if type(cofactor_names[i]) == str else parsed_cofactor_df['id'][i] for i in range(len(cofactor_names))]
pathway_names = [pathway_names[i] if type(pathway_names[i]) == str else parsed_pathway_df['id'][i] for i in range(len(pathway_names))]

# save C, P and E to julia-compatible formats
np.savetxt('notebooks/cofactors/data/C_matrix.csv', C.astype(np.int64), delimiter=',', fmt='%i')
np.savetxt('notebooks/cofactors/data/P_matrix.csv', P.astype(np.int64), delimiter=',', fmt='%i')
np.savetxt('notebooks/cofactors/data/E_matrix.csv', E.astype(np.int64), delimiter=',', fmt='%i')
np.savetxt('notebooks/cofactors/data/W_matrix.csv', W.astype(np.float64), delimiter=',')
np.savetxt('notebooks/cofactors/data/W1_matrix.csv', W1.astype(np.float64), delimiter=',')
np.savetxt('notebooks/cofactors/data/W2_matrix.csv', W2.astype(np.float64), delimiter=',')
np.savetxt('notebooks/cofactors/data/A_matrix.csv', A.astype(np.float64), delimiter=',')


# write all ids to single file with each list on a new line
with open('notebooks/cofactors/data/complex_ids.txt', 'w') as f:
    f.write('\n'.join(complex_ids))
with open('notebooks/cofactors/data/protein_ids.txt', 'w') as f:
    f.write('\n'.join(protein_ids))
with open('notebooks/cofactors/data/cofactor_ids.txt', 'w') as f:
    f.write('\n'.join(cofactor_ids))
with open('notebooks/cofactors/data/element_ids.txt', 'w') as f:
    f.write('\n'.join(element_ids))
with open('notebooks/cofactors/data/pathway_ids.txt', 'w') as f:
    f.write('\n'.join(pathway_ids))
with open('notebooks/cofactors/data/amino_acid_ids.txt', 'w') as f:
    f.write('\n'.join(amino_acid_ids))

# write all names to single file with each list on a new line
with open('notebooks/cofactors/data/complex_names.txt', 'w') as f:
    f.write('\n'.join(complex_names))
with open('notebooks/cofactors/data/protein_names.txt', 'w') as f:
    f.write('\n'.join(protein_names))
with open('notebooks/cofactors/data/cofactor_names.txt', 'w') as f:
    f.write('\n'.join(cofactor_names))
with open('notebooks/cofactors/data/pathway_names.txt', 'w') as f:
    f.write('\n'.join(pathway_names))


# save counts of proteins and complexes
np.savetxt('notebooks/cofactors/data/counts.csv', np.array(counts, dtype=np.int64), delimiter=',', fmt='%i')

# save counts of proteins and complexes
np.savetxt('notebooks/cofactors/data/rich_counts.csv', np.array(rich_counts, dtype=np.int64), delimiter=',', fmt='%i')


# save masses
np.savetxt('notebooks/cofactors/data/monomer_masses.csv', np.array(parsed_protein_df['sequence_mass']), delimiter=',', fmt='%f')

# save tree_matrix
np.savetxt('notebooks/cofactors/data/tree_matrix.csv', tree_matrix, delimiter=',', fmt='%i')



In [ ]:
# save counts as csv with header for Khoa
counts.to_csv('notebooks/cofactors/data/counts_header.csv', index=False)

parsed_protein_df.to_csv('notebooks/cofactors/data/protein_table.csv', index=False)

# Tests

In [ ]:
parsed_protein_df[parsed_protein_df['id'] == 'EG12332-MONOMER']

First, let's test that the cofactor counts are correct for SULFITE-REDUCT-CPLX and its constituent BETACOMP-MONOMER

In [ ]:
for i in range(len(parsed_protein_df)):
    sequence = parsed_protein_df.at[i, 'sequence_processed']

    if type(sequence) is dict and 'U' in sequence:
        print(f'found U in {parsed_protein_df.at[i, "id"]}')

In [ ]:
print(f'Sulfite reductase complex count: {counts[1020]}, hemoprotein subunit count: {counts[1271]}')
print(f'Expected iron count: {counts[1020] * 5 * 4} for the first, {counts[1271] * 5 } for the second')

Let's check the pathway counts for the related pathway.

In [ ]:
cplx_pathways = pathway_ids[ np.where((C @ W)[1020, :])[0][0] ]
monomer_pathways = pathway_ids[ np.where((C @ W)[1271, :])[0][0] ]


print(f'Sulfite reductase is in the following pathways: {cplx_pathways}, while the hemoprotein is in {monomer_pathways}')

That means the pathway counts for iron should be the same as the cplx counts (or larger)

In [ ]:
# pathway_counts = (iron_counts @ C_n @ W)[  np.where((C @ W)[1020, :])[0][0]  ]

# print(f'Pathway counts for iron: {pathway_counts}')

In [ ]:
print(f"""Monomer index: {C_names.index('SUPEROX-DISMUTFE-MONOMER')}, complex index: {C_names.index('SUPEROX-DISMUTFE-CPLX')}""")

In [ ]:
print(f"""Monomer count: {counts[5232]}, complex count: {counts[1021]}""")

In [ ]:
print(f"""Monomer index: {C_names.index('SUPEROX-DISMUTMN-MONOMER')}, complex index: {C_names.index('SUPEROX-DISMUTMN-CPLX')}""")
print(f"""Monomer count: {counts[5233]}, complex count: {counts[1022]}""")

In [ ]:
mn_idx = element_ids.index('MN')
mn_counts = (np.array(counts).reshape(-1, 1) * C @ P @ E)[:, mn_idx]
# each complex count of 1 counts as 1 total annotation, so need to normalize
C_n = C / (C.sum(axis=1, keepdims=1) + 1e-10)

nz_idx = np.where((C_n @ W @ W2).T @ mn_counts)[0]
mn_counts_group = (C_n @ W @ W2).T @ mn_counts

In [ ]:
cplx_pathways = pathway_ids[ np.where((C @ W)[5233, :])[0][0] ]
monomer_pathways = pathway_ids[ np.where((C @ W)[1022, :])[0][0] ]


print(f'Superoxide dismutase is in the following pathways: {cplx_pathways}, while the monomer is in {monomer_pathways}')

In [ ]:
pathway_counts = (mn_counts @ C_n @ W @ W2)[  np.where((C @ W @ W2)[5233, :])[0][0]  ]
pathway_counts

In [ ]:
mn_counts[1022]

In [ ]:
counts

In [ ]:
parsed_pathway_df[parsed_pathway_df['id'] == 'PWY0-1507']

In [ ]:
pathway_ids.index('PWY0-1507')

In [ ]:
np.where(W2[753,:])

In [ ]:
pathway_ids[718]

## go annotation for proteins

In [ ]:
parsed_protein_df[parsed_protein_df['id'] == 'BCCP-MONOMER']

In [ ]:
parsed_protein_df[parsed_protein_df['id'] == 'EG10889-MONOMER']

In [ ]:
parsed_protein_df

In [ ]:
bulk.loc[:, 'RNA0-301[c]']

In [ ]:
bulk.loc[:, 'NARH-MONOMER[j]']

In [ ]:
ecocyc_

In [ ]:
bulk.loc[:, 'NITRATREDUCTA-CPLX[i]']

In [ ]:
for element in fba['estimated_fluxes']:
    print(len(element))

In [ ]:
fluxes

In [ ]:
stoichiometry.shape

In [ ]:
vars(metabolism).keys()

In [ ]:
metabolism.reaction_names.index('RXN0-7077')

In [ ]:
fluxes[:, 3470]

In [ ]:
time = '2600'
date = '2023-11-28'
experiment = 'metabolism-redux-classic'
entry = f'{experiment}_{time}_{date}'
folder = f'out/cofactors/{entry}/'

f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry

In [ ]:
output_all = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output_all['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

fluxes = np.array(fba['estimated_fluxes'][1:])
exchanges = fba['estimated_exchange_dmdt']

output['listeners']['unique_molecule_counts']['active_ribosome']

In [ ]:
flux_df = pd.DataFrame(fluxes, columns=metabolism.reaction_names)
flux_df

# find top 10 fluxes for timestep 50
flux_df.loc[50, :].sort_values(ascending=False)[0:50]

In [ ]:
exchanges.keys()

In [ ]:
exch_df_exp = pd.DataFrame([exchanges['GLC[p]'], exchanges['OXYGEN-MOLECULE[p]']]).T
exch_df_exp.columns = ['GLC[p]', 'OXYGEN-MOLECULE[p]']
exch_df_exp.to_csv('notebooks/cofactors/data/exchanges.csv', index=False)

In [ ]:
flux_df.loc[50, 'NADH-DEHYDROG-A-RXN-NADH/UBIQUINONE-8/PROTON//NAD/CPD-9956/PROTON.46.']

In [ ]:
for name in metabolism.reaction_names:
    if 'RXN0-5388' in name:
        print(name)